In [ ]:
/* ===================================================================================
   METRIC: EMP01 - Canadian Officer / PEP Presence
   
   BUSINESS QUESTION: 
   Does an Assessable Unit contain at least one active Canadian officer 
   (Politically Exposed Person / PEP)?
   
   LOGIC SUMMARY:
   1. Filters the Employee PEP List to only include Region = 'Canada'.
   2. Cleans the Cost Center string to match the Master Mapping format.
   3. Joins the Canadian officers to the Master Mapping table.
   4. Rolls up the data by Assessable Unit. If 1 or more Canadian officers 
      are linked to that Unit, the flag outputs 'Yes'. Otherwise, 'No'.
=================================================================================== */

WITH Canadian_PEP AS (
    SELECT 
        -- Grabs the pure ID before the space (e.g., "0533 OTTAWA" -> "0533")
        SUBSTRING_INDEX(TRIM(Costcenter), ' ', 1) AS Extracted_CC
    FROM hive_metastore.ra_adido_2025.employee_pep_list_as_of_oct312025
    WHERE TRIM(Region) = 'Canada'
)

SELECT 
    m.`Assessable Unit ID and Name` AS `Assessable Unit`,
    
    -- If the join found at least one match, flag 'Yes'. Otherwise 'No'.
    CASE 
        WHEN COUNT(p.Extracted_CC) > 0 THEN 'Yes' 
        ELSE 'No' 
    END AS `Has Canadian Officer?`

FROM cost_center m

LEFT JOIN Canadian_PEP p
    -- Smart Join: Adds a leading '0' only if the Mapping ID is exactly 3 characters
    ON CASE 
        WHEN LENGTH(TRIM(CAST(m.`Cost Center` AS STRING))) = 3 
        THEN CONCAT('0', TRIM(CAST(m.`Cost Center` AS STRING))) 
        ELSE TRIM(CAST(m.`Cost Center` AS STRING)) 
       END = p.Extracted_CC

WHERE m.`Assessable Unit ID and Name` IS NOT NULL

GROUP BY 
    m.`Assessable Unit ID and Name`

In [ ]:
%run ./00_Mapping_Setup

In [ ]:
/* ===================================================================================
   METRIC: EMP01 - Flattened Canadian Officer Presence
   
   BUSINESS QUESTION: 
   Which specific Assessable Units (Primary AND Additional) contain at least 
   one active Canadian officer? 
=================================================================================== */

WITH Canadian_PEP AS (
    SELECT DISTINCT SUBSTRING_INDEX(TRIM(Costcenter), ' ', 1) AS Extracted_CC
    FROM hive_metastore.ra_adido_2025.employee_pep_list_as_of_oct312025
    WHERE TRIM(Region) = 'Canada'
)

SELECT DISTINCT
    m.Raw_CC AS `Cost Center`,
    m.AU_ID AS `Assessable Unit ID`,
    m.AU_Name AS `Assessable Unit Name`,
    m.Segment AS `Segment`,
    'Yes' AS `Has Canadian Officer?`

FROM vw_flattened_mapping m

INNER JOIN Canadian_PEP p
    -- Smart Join to handle 3-digit vs 4-digit Cost Centers
    ON CASE 
        WHEN LENGTH(m.Raw_CC) = 3 THEN CONCAT('0', m.Raw_CC) 
        ELSE m.Raw_CC 
       END = p.Extracted_CC;